In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
df.head()

In [ ]:
df.Outcome.value_counts()

In [ ]:
df[(df.Glucose > 160) & (df.Outcome == 0)].Outcome.count()

On regarde s'il y a des données manquantes

In [ ]:
df.count()

In [ ]:
df[df.BloodPressure == 0].BloodPressure.count()

In [ ]:
df[df.Glucose == 0].BloodPressure.count()

In [ ]:
df[df.BMI == 0].BloodPressure.count()

In [ ]:
df[df.SkinThickness == 0].BloodPressure.count()

Il n'y pas de valeur de type NaN mais on remarque qu'il y a des données égales à 0 qui ne devrait pas être nulles.
Comme **BloodPressure**, **SkinThickness** et **Insulin**.

# Affichage des graphes

In [ ]:
sns.boxplot(x='Outcome', y='Glucose', data=df)

In [ ]:
fig = sns.FacetGrid(df, hue="Outcome", aspect=3, palette="Set2")
fig.map(sns.kdeplot, "Glucose", shade=True)
fig.add_legend()

In [ ]:
sns.lmplot(x="SkinThickness", y="Glucose", data=df, fit_reg=False, hue="Outcome")

In [ ]:
sns.pairplot(df, hue="Outcome")

On remarque bien que le Glucose est l'élément le plus impactant ici.

In [ ]:
df.columns

*Pregnancies* ne semble pas être un élément important pour la prédicition

In [ ]:
data = df.drop(['Pregnancies', 'BloodPressure'], axis=1)

In [ ]:
data.count()

## Données manquantes

In [ ]:
data[data.Glucose != 0].Glucose.mean()

### Complétion des valeurs nulles

In [ ]:
def replace_0(df,col) :
    df1 = df.copy()
    n = df.shape[0]
    m = df[col].mean()
    s = df[col].std()
    for i in range(n) :
        if df.loc[i,col]==0 :
            df1.loc[i,col] = df[df[col] != 0][col].mean()
    return df1

In [ ]:
data = replace_0(data, 'Glucose')
#data = replace_0(data, 'Insulin')
#data = replace_0(data, 'BloodPressure')


In [ ]:
data.head()

## Prédiction

In [ ]:
X = data.drop(['Outcome'], axis=1)
y = data.Outcome

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

### Regression Logique

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
y_lr = lr.predict(X_test)

# Mesure de perf

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, auc, accuracy_score

In [ ]:
print(confusion_matrix(y_test, y_lr))

In [ ]:
print(accuracy_score(y_test,y_lr))

## Arbre de décisions

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_dtc = dtc.predict(X_test)
print(accuracy_score(y_test, y_dtc))

In [ ]:
plt.figure(figsize=(30,30))
tree.plot_tree(dtc, feature_names=X_train.columns, class_names=['Negatif','Positif'], fontsize=14, filled=True)

## Foret

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

In [ ]:
print(confusion_matrix(y_test, y_rf))

In [ ]:
print(classification_report(y_test, y_rf))